In [ ]:
import json
import os
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

In [ ]:
class TimeSeriesTransformerEncoderDecoder(nn.Module):
    def __init__(
            self,
            input_dim=6,
            seq_len=50,
            d_model=64,
            nhead=4,
            num_layers=2,
            dim_feedforward=128,
            dropout=0.1,
    ):
        super().__init__()

        self.input_dim = input_dim
        self.seq_len = seq_len
        self.d_model = d_model

        self.input_embedding = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len=seq_len)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        
        self.output_layer = nn.Linear(d_model, input_dim)

    def forward(self, src):
        src = src.transpose(1, 2)  # [batch_size, seq_len, input_dim]
        src_embedded = self.input_embedding(src)
        src_embedded = self.pos_encoder(src_embedded)
        
        memory = self.transformer_encoder(src_embedded)
        output = self.transformer_decoder(memory, memory)  # tgt = memory pour reconstruction
        output = self.output_layer(output)
        output = output.transpose(1, 2)  # [batch_size, input_dim, seq_len]
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [ ]:
# All windows

data = pd.read_csv("./final_stocks_2.csv", usecols=["log_return_DlyClose", "log_return_DlyLow", "log_return_DlyHigh", "log_return_DlyBid", "log_return_DlyAsk", "volume_normalized"])

In [ ]:
# Positive Future windows

data = pd.read_csv("./final_stocks_4.csv", usecols=["DlyClose", "DlyLow", "DlyHigh", "DlyBid", "DlyAsk", "DlyVol"])

In [ ]:
# Negative Future windows

data = pd.read_csv("./final_stocks_negative.csv", usecols=["DlyClose", "DlyLow", "DlyHigh", "DlyBid", "DlyAsk", "DlyVol"])

In [ ]:
seq_length = 50

In [ ]:
# Load data to train

stride = seq_length
windows = np.array([data[i:i + seq_length] for i in range(0, len(data) - seq_length - 1, stride)])
windows = torch.FloatTensor(windows).transpose(1, 2)
train_size = int(0.9 * len(windows))
val_size = len(windows) - train_size
train_data, test_data = random_split(windows, [train_size, val_size])
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

In [ ]:
# Création des modèles
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32
seq_len = 50
input_dim = 6
d_model = 64
n_head=4
num_layers=2
dim_feedforward=128
epochs = 10
learning_rate = 0.001
latent_dim = d_model

model_folder = "models/transformer_encoder_decoder_negative/"

model = TimeSeriesTransformerEncoderDecoder(
    input_dim=input_dim,
    seq_len=seq_len,
    d_model=d_model,
    nhead=n_head,
    num_layers=num_layers,
    dim_feedforward=dim_feedforward,
    dropout=0.1
).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
timer = time.time()
folder = model_folder + str(timer) + "/"
os.makedirs(folder + "checkpoints/", exist_ok=True)

# Sauvegarde de la configuration
config = {
    # Paramètres d'entraînement
    'batch_size': batch_size,
    'epochs': epochs,
    'learning_rate': learning_rate,
    'device': str(device),
    
    # Paramètres du modèle
    'input_dim': input_dim,
    'seq_len': seq_length,
    'num_layers': num_layers,
    'dim_feedforward': dim_feedforward,
    'n_head': n_head,
    'd_model': d_model,
    
    # Informations sur l'architecture
    'optimizer': optimizer.__class__.__name__,
    
    # Timestamp et dossier
    'timestamp': timer,
    'model_folder': model_folder
}

# Sauvegarde de la configuration
with open(folder + 'config.json', 'w', encoding='utf-8') as f:
    json.dump(config, f, ensure_ascii=False, indent=4)

In [ ]:
from src.utils.train import train_model

train_model(model, 100, train_loader, test_loader, optimizer, device, folder)

In [ ]:
timestamp = "1740400132.6410854"
path_checkpoint = model_folder + timestamp + "/checkpoints/model_epoch_11.pt"

In [ ]:
checkpoint = torch.load(path_checkpoint)
model.load_state_dict(checkpoint['transformer_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

In [ ]:
from src.types import dsf_dtype_dict

dsf = pd.read_csv("./dsf_v2_patched_small.csv",
                  dtype=dsf_dtype_dict,
                  parse_dates=['DlyCalDt'],
                  usecols=['DlyCalDt', 'PERMNO', 'DlyClose']
                  )

In [ ]:
# ALL WINDOWS
final_stocks = pd.read_csv("./final_stocks_2.csv")

In [ ]:
from src.benchmark.benchmark import build_predictions

predictions_results = build_predictions(model, final_stocks, device, model_folder + timestamp + "/")

In [ ]:
predictions_results = pd.read_csv(folder + "predictions_results.csv")

In [ ]:
from src.benchmark.benchmark import build_quantiles

quantiles = [0.98, 0.99, 0.994, 0.995, 0.996, 0.997]
quantiles_results = build_quantiles(predictions_results, quantiles, dsf)

In [ ]:
from src.benchmark.benchmark import analyze_quantiles

analyze_quantiles(quantiles_results, model_folder + timestamp + "/stats", False)
analyze_quantiles(quantiles_results, model_folder + timestamp + "/stats", True)